In [1]:
import glob
import re
import json
import itertools
import pandas as pd
from pathlib import Path
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Lambda

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from tqdm import tqdm

from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import TimeSeriesSplit

# all the functions from helpers.py
from helpers_scenario2 import *

In [2]:
scenario_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/"
fold = 0

phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)

In [3]:
annotations_folder = '../data/raw/scenario_2/fold_0/train/annotations/'
physiology_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology/" 
zipped_files = zip_csv_files(annotations_folder, physiology_folder)

subjects, videos = get_subs_vids(physiology_folder)

splits = split_subjects_train_test(subjects, 3)

In [4]:
import json
from keras.layers import Dropout, LSTM
from keras.regularizers import l1, l2
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from kerastuner.tuners import RandomSearch

from keras.layers import MaxPooling1D

import json
from keras.layers import Dropout, LSTM
from keras.regularizers import l1, l2
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from kerastuner.tuners import RandomSearch

from keras.layers import MaxPooling1D

def create_cnn_lstm_model(input_shape, lstm_units=64, dropout_rate=0.4, kernel_regularizer_l1=0.0032):
    input_signal = Input(shape=input_shape)

    x = Conv1D(16, 5, padding='same')(input_signal)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = MaxPooling1D(pool_size=2)(x)

    x = LSTM(lstm_units)(x)

    x = Dense(32, activation='relu', kernel_regularizer=l1(kernel_regularizer_l1))(x)

    return Model(inputs=input_signal, outputs=x)

C:\Users\xochipilli\AppData\Local\Temp\ipykernel_8464\1364739470.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [5]:
import pandas as pd
from pathlib import Path


from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
import joblib
from joblib import Parallel, delayed, parallel_backend
import multiprocessing
from multiprocessing import Value
from tqdm.auto import tqdm
import itertools


from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import TimeSeriesSplit

# all the functions from helpers.py
from helpers_scenario2 import *

In [6]:
scenario_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/"
fold = 0

phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)
annotations_folder = '../data/raw/scenario_2/fold_0/train/annotations/'
physiology_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology/" 
zipped_files = zip_csv_files(annotations_folder, physiology_folder)

subjects, videos = get_subs_vids(physiology_folder)

num_cpu_cores = multiprocessing.cpu_count()

def process_files(annotation_file, physiology_file,):
    df_annotations = pd.read_csv(annotation_file)
    df_physiology = pd.read_csv(physiology_file)
    
    print(physiology_file)
    X, y, numeric_column_indices = preprocess(df_physiology, df_annotations, predictions_cols=['arousal','valence'], aggregate=None, window=[-10000, 10000], partition_window = 1)
    # print(X.shape, y.shape)
    
    save_files(X, y, annotation_file, phys_folder, ann_folder)
    
    return None

## Process the files using the context manager
#with parallel_backend('multiprocessing', n_jobs=num_cpu_cores // 3):
#    with tqdm_joblib(tqdm(total=len(zipped_files), desc="Files", leave=False)) as progress_bar:
#        results = Parallel()(
#            (delayed(process_files)(ann_file, phys_file) for ann_file, phys_file in zipped_files)
#        )
#
for ann_file, phys_file in zipped_files:
    process_files(ann_file, phys_file,)
    


../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology\sub_0_vid_0.csv
../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology\sub_0_vid_10.csv
../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology\sub_0_vid_11.csv
../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology\sub_0_vid_13.csv
../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology\sub_0_vid_14.csv
../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology\sub_0_vid_2.csv


In [40]:
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
import threading
import multiprocessing
from tqdm import tqdm
import itertools
import json

# Initialize an empty dictionary to store the fold results
fold_results = {}

# Loop through the splits
for split_index, split in enumerate(splits):

        X_train, X_test, y_train, y_test, numeric_column_indices, categorical_column_indices = preprocess(
            df_physiology_video.copy(), df_annotations_video.copy(), split=split, predictions_cols=['arousal', 'valence'], aggregate=None,
            window_duration=10000, resample_rate=100)

        print("X_test shape:", X_test.shape)
        print("y_test shape:", y_test.shape)
                
        # Extract arousal and valence values from y_train and y_test
        y_arousal_train = y_train[:, 0]
        y_valence_train = y_train[:, 1]
        y_arousal_test = y_test[:, 0]
        y_valence_test = y_test[:, 1]

        # Normalize the input data
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        input_shape = (X_train.shape[1], X_train.shape[2])

        # Define a Keras wrapper for the model
        def model_builder(hp):
            lstm_units = hp.Int('lstm_units', min_value=32, max_value=128, step=32)
            dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
            kernel_regularizer_l1 = hp.Choice('kernel_regularizer_l1', values=[0.001, 0.01, 0.1])

            model = create_cnn_lstm_model(input_shape, lstm_units, dropout_rate, kernel_regularizer_l1)

            valence_output = Dense(1, activation='sigmoid')(model.output)
            valence_output = Lambda(lambda x: x * 8 + 1, name='valence_output')(valence_output)
            
            arousal_output = Dense(1, activation='sigmoid')(model.output)
            arousal_output = Lambda(lambda x: x * 8 + 1, name='arousal_output')(arousal_output)

            # Combine the model
            final_model = Model(inputs=model.input, outputs=[valence_output, arousal_output])

            # Compile the model
            final_model.compile(optimizer='adam',
                                loss={'valence_output': 'mse',
                                    'arousal_output': 'mse'},
                                metrics=[tf.keras.metrics.RootMeanSquaredError()])

            return final_model

        # Hyperparameter search
        tuner = RandomSearch(
            model_builder,
            objective='val_loss',
            max_trials=5,
            executions_per_trial=3,
            directory='random_search',
            project_name='emotion_recognition'
        )

        early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

        tuner.search(X_train, {'valence_output': y_valence_train, 'arousal_output': y_arousal_train},
                    validation_split=0.2, epochs=1, batch_size=32,
                    callbacks=[early_stopping_callback])

        # Get the best model
        best_model = tuner.get_best_models(num_models=1)[0]

        # Train the best model
        history = best_model.fit(X_train, {'valence_output': y_valence_train, 'arousal_output': y_arousal_train},
                                validation_split=0.2, epochs=50, batch_size=32,
                                callbacks=[early_stopping_callback])

        evaluation = best_model.evaluate(X_test, [y_valence_test, y_arousal_test])

        # Save the evaluation results in the dictionary
        key = f"video_{video}_split_{split_index}"
        results[key] = {
            'valence_output_root_mean_squared_error': evaluation[3],
            'arousal_output_root_mean_squared_error': evaluation[4]
        }

    # Calculate the average of the evaluation results
    evaluation_sum = np.array([list(rmse_dict.values()) for rmse_dict in results.values()]).sum(axis=0)
    average_evaluation = evaluation_sum / len(splits)

    # Save the results for the current video
    video_results[f"video_{video}"] = results

    # Save the average evaluation in the dictionary
    video_results[f"video_{video}_average"] = {
        'valence_output_root_mean_squared_error': average_evaluation[0],
        'arousal_output_root_mean_squared_error': average_evaluation[1]
    }

    # Reset the results dictionary for the next video
    results = {}


Processing videos:   0%|          | 0/8 [00:00<?, ?video/s]

Processing video: 0
X_test shape: (20543, 1000, 15)
y_test shape: (20543, 2)
642/642 [==============================] - 5s 7ms/step - loss: 15.0058 - valence_output_loss: 8.2676 - arousal_output_loss: 6.1055 - valence_output_root_mean_squared_error: 2.8753 - arousal_output_root_mean_squared_error: 2.4709
X_test shape: (20542, 1000, 15)
y_test shape: (20542, 2)
642/642 [==============================] - 8s 12ms/step - loss: 16.2454 - valence_output_loss: 9.0707 - arousal_output_loss: 6.6110 - valence_output_root_mean_squared_error: 3.0118 - arousal_output_root_mean_squared_error: 2.5712
X_test shape: (20543, 1000, 15)
y_test shape: (20543, 2)
642/642 [==============================] - 7s 11ms/step - loss: 6.7567 - valence_output_loss: 2.2828 - arousal_output_loss: 3.7566 - valence_output_root_mean_squared_error: 1.5109 - arousal_output_root_mean_squared_error: 1.9382


Processing videos:  12%|█▎        | 1/8 [02:14<15:42, 134.61s/video]

Processing video: 10
X_test shape: (12240, 1000, 15)
y_test shape: (12240, 2)
383/383 [==============================] - 4s 9ms/step - loss: 8.1449 - valence_output_loss: 2.6057 - arousal_output_loss: 4.8690 - valence_output_root_mean_squared_error: 1.6142 - arousal_output_root_mean_squared_error: 2.2066
X_test shape: (12240, 1000, 15)
y_test shape: (12240, 2)
383/383 [==============================] - 3s 8ms/step - loss: 25.9670 - valence_output_loss: 10.7590 - arousal_output_loss: 14.6302 - valence_output_root_mean_squared_error: 3.2801 - arousal_output_root_mean_squared_error: 3.8249
X_test shape: (12240, 1000, 15)
y_test shape: (12240, 2)
383/383 [==============================] - 3s 8ms/step - loss: 22.0859 - valence_output_loss: 7.9833 - arousal_output_loss: 13.1590 - valence_output_root_mean_squared_error: 2.8255 - arousal_output_root_mean_squared_error: 3.6275


Processing videos:  25%|██▌       | 2/8 [03:26<09:47, 97.85s/video] 

...

Processing video: 14
X_test shape: (14640, 1000, 15)
y_test shape: (14640, 2)
458/458 [==============================] - 5s 10ms/step - loss: 4.7233 - valence_output_loss: 0.9734 - arousal_output_loss: 3.3380 - valence_output_root_mean_squared_error: 0.9866 - arousal_output_root_mean_squared_error: 1.8270
X_test shape: (14640, 1000, 15)
y_test shape: (14640, 2)
458/458 [==============================] - 4s 9ms/step - loss: 30.5373 - valence_output_loss: 18.0640 - arousal_output_loss: 11.9215 - valence_output_root_mean_squared_error: 4.2502 - arousal_output_root_mean_squared_error: 3.4528
X_test shape: (14640, 1000, 15)
y_test shape: (14640, 2)
458/458 [==============================] - 5s 10ms/step - loss: 2.7560 - valence_output_loss: 0.1176 - arousal_output_loss: 2.0258 - valence_output_root_mean_squared_error: 0.3429 - arousal_output_root_mean_squared_error: 1.4233


Processing videos:  62%|██████▎   | 5/8 [08:20<05:02, 100.76s/video]

Processing video: 2
X_test shape: (12024, 1000, 15)
y_test shape: (12024, 2)
376/376 [==============================] - 4s 9ms/step - loss: 22.5493 - valence_output_loss: 12.5103 - arousal_output_loss: 9.5032 - valence_output_root_mean_squared_error: 3.5370 - arousal_output_root_mean_squared_error: 3.0827
X_test shape: (12023, 1000, 15)
y_test shape: (12023, 2)
376/376 [==============================] - 6s 16ms/step - loss: 4.6532 - valence_output_loss: 2.4710 - arousal_output_loss: 1.6217 - valence_output_root_mean_squared_error: 1.5719 - arousal_output_root_mean_squared_error: 1.2735
X_test shape: (12023, 1000, 15)
y_test shape: (12023, 2)
376/376 [==============================] - 6s 15ms/step - loss: 24.4146 - valence_output_loss: 14.4553 - arousal_output_loss: 8.8901 - valence_output_root_mean_squared_error: 3.8020 - arousal_output_root_mean_squared_error: 2.9816


Processing videos:  75%|███████▌  | 6/8 [10:32<03:43, 111.65s/video]

Processing video: 20
X_test shape: (18715, 1000, 15)
y_test shape: (18715, 2)
585/585 [==============================] - 8s 14ms/step - loss: 23.2956 - valence_output_loss: 10.5440 - arousal_output_loss: 12.2675 - valence_output_root_mean_squared_error: 3.2472 - arousal_output_root_mean_squared_error: 3.5025
X_test shape: (18716, 1000, 15)
y_test shape: (18716, 2)
585/585 [==============================] - 7s 12ms/step - loss: 19.6993 - valence_output_loss: 18.7427 - arousal_output_loss: 0.5141 - valence_output_root_mean_squared_error: 4.3293 - arousal_output_root_mean_squared_error: 0.7170
X_test shape: (18715, 1000, 15)
y_test shape: (18715, 2)
772/936 [=======================>......] - ETA: 7s - loss: 6.7447 - valence_output_loss: 3.1402 - arousal_output_loss: 0.8857 - valence_output_root_mean_squared_error: 1.7721 - arousal_output_root_mean_squared_error: 0.9411

In [21]:
print("Results for each split:")
print(results)

Results for each split:
{'video_0_subject_13_split_0': [409.2861328125, 232.13124084472656, 176.3783416748047, 15.235854148864746, 13.28075122833252], 'video_0_subject_34_split_1': [9.059562683105469, 4.963016510009766, 2.896031618118286, 2.227782964706421, 1.7017730474472046], 'video_0_subject_43_split_2': [30.761016845703125, 7.859403133392334, 21.74477767944336, 2.8034627437591553, 4.663129806518555], 'average': array([149.70223745,  81.65122016,  67.00638366,   6.75569995,
         6.54855136])}


In [18]:
print(final_model.evaluate(X_test, [y_valence_test, y_arousal_test]))

642/642 [==============================] - 5s 8ms/step - loss: 30.7610 - valence_output_loss: 7.8594 - arousal_output_loss: 21.7448 - valence_output_root_mean_squared_error: 2.8035 - arousal_output_root_mean_squared_error: 4.6631
[30.761016845703125, 7.859403133392334, 21.74477767944336, 2.8034627437591553, 4.663129806518555]


In [ ]:
print("X_train shape:", X_train.shape)
print("y_valence_train shape:", y_valence_train.shape)
print("y_arousal_train shape:", y_arousal_train.shape)
print("X_test shape:", X_test.shape)
print("y_valence_test shape:", y_valence_test.shape)
print("y_arousal_test shape:", y_arousal_test.shape)

X_train shape: (1000, 41085, 15)
y_valence_train shape: (41085,)
y_arousal_train shape: (41085,)
X_test shape: (1000, 20543, 15)
y_valence_test shape: (20543,)
y_arousal_test shape: (20543,)


In [22]:
[split for split_index, split in enumerate(splits)]

[{'train': [34, 43, 31, 45, 32, 9, 2, 11, 27, 26, 18, 28, 7, 29, 5, 41],
  'test': array([13,  0,  1, 20, 17, 15, 30, 24], dtype=int64)},
 {'train': [13, 43, 0, 45, 1, 9, 20, 11, 17, 26, 15, 28, 30, 29, 24, 41],
  'test': array([34, 31, 32,  2, 27, 18,  7,  5], dtype=int64)},
 {'train': [13, 34, 0, 31, 1, 32, 20, 2, 17, 27, 15, 18, 30, 7, 24, 5],
  'test': array([43, 45,  9, 11, 26, 28, 29, 41], dtype=int64)}]

In [15]:
X_test = np.swapaxes(X_test, 0, 1)

In [16]:
print(final_model.evaluate(X_test, [y_valence_test, y_arousal_test]))


ValueError: in user code:

    File "c:\Users\xochipilli\Anaconda3\lib\site-packages\keras\engine\training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\xochipilli\Anaconda3\lib\site-packages\keras\engine\training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\xochipilli\Anaconda3\lib\site-packages\keras\engine\training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\xochipilli\Anaconda3\lib\site-packages\keras\engine\training.py", line 1665, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\xochipilli\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\xochipilli\Anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueErro

In [14]:
print(X_test.shape)
print(y_valence_test.shape)
print(y_arousal_test.shape)


(1000, 20543, 15)
(20543,)
(20543,)
